In [50]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *
from einops import rearrange

In [54]:
import torch
import torch.nn as nn
import pandas as pd 

In [55]:
x0 = 0.0 # initial condition
sigma = 1 # volatility

segs=5
d=20
r = 0.0 # risk free rate
batch_size = 100 # batch size
steps=200
T = 1 # maturity
dt = T/steps # mesh size

dt = T/steps # mesh size
dt_new = T/segs # new mesh after shrinkage
level = 3 # truncation level

In [56]:
batch_in=100
MOMENTUM = 0.99
EPSILON = 1e-6
import warnings
warnings.filterwarnings("ignore")

In [57]:
def create_stock(x0,r,sigma,T,steps,n_path,dW):  
    s_vec=[];
    w0=np.ones((n_path,1,d))*1e-6
    dW=np.concatenate((w0,dW),axis=1) ## the first time slot is then 0
    s_vec.append(np.ones((n_path,d))*x0)
    for i in range(steps): 
        s_vectemp=s_vec[-1]+ r*s_vec[-1]*dt+ dW[:,i+1,:]*sigma
        s_vec.append(s_vectemp)
    BM_path=np.cumsum(dW,axis=1) ## find the cumulative sum
    S_path=rearrange(np.array(s_vec), 'b c h -> c b h') 
    return BM_path, S_path

def jointime(T,path): 
    n_path, steps, d=path.shape
    dt=T/(steps-1); 
    
    times=np.arange(0,T,dt) ## This can be taken out
    times=np.append(times,T); 
    times_vec=np.tile(times,[1,1]); 
    times_vec=np.transpose(times_vec)
    times_vec=np.tile(times_vec,[n_path,1,1])
    times_vec=np.concatenate((times_vec, path),axis=2)
    return times_vec

def ComputeMultiLevelSig(path, number_of_segment, depth,log_sig=False):
    n_batch, nsteps, chanels = path.shape
    t_vec = np.arange(0, nsteps-1, int(nsteps / number_of_segment))
    t_vec = np.append(t_vec, nsteps-1)
    MultiLevelSig = []
    s=iisig.prepare(d+1,depth)
    
    if log_sig: 
        ll=iisig.logsig(np.expand_dims(path[:,0,:],axis=1),s)
        MultiLevelSig.append(ll)
        for i in range(len(t_vec)-1):    
        ## Notice that we only use the signature of the concatenation of time and space.
            MultiLevelSig.append(iisig.logsig(path[:,0:t_vec[i+1]+1,:],s)) ##if not
        MultiLevelSig=np.stack(MultiLevelSig)  
        MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
    else: 
        ll=iisig.sig(np.expand_dims(path[:,0,:],axis=1),depth)
        MultiLevelSig.append(ll)
        for i in range(len(t_vec)-1):    
        ## Notice that we only use the signature of the concatenation of time and space.
            MultiLevelSig.append(iisig.sig(path[:,0:t_vec[i+1]+1,:],depth)) ##if not
            #MultiLevelSig.append(path_class.signature(t_vec[i],t_vec[i+1]+1))
        MultiLevelSig=np.stack(MultiLevelSig)  
        MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
        
    return MultiLevelSig

In [58]:
def generate_samples(batch_in=100):
    """
    Produce the signature, dW for computation, YT the terminal condition
    x_ten, selection the path and location to be used
    """
    dW=np.sqrt(dt)*np.random.normal(size=(batch_in, steps,d))
    
    pth2=create_stock(x0,r,sigma,T,steps,batch_in,dW)
    BM_timePath=jointime(T,pth2[0]); 
    S_timePath=jointime(T,pth2[1]);
    
    sigs=ComputeMultiLevelSig(S_timePath, segs, 3,True)
    selection = np.linspace(0,steps, segs+1, dtype = np.int)

    BM_seg=BM_timePath[:,selection,1:]
    dW=BM_seg[:,1:,:]-BM_seg[:,:-1,:]

    dW=torch.tensor(dW,dtype=torch.float32)
    sigs=torch.tensor(sigs,dtype=torch.float32)
    S_timePath=torch.tensor(S_timePath,dtype=torch.float32)
    
    x_ten=S_timePath[:,:,1:]
    
    YT=terminal_f(x_ten)
    
    return sigs, dW, YT, x_ten, selection[:-1]

In [59]:
def terminal_f(x_ten):
    res=(torch.sum(x_ten[:,:-1,:],dim=(1,2))*dt)**2
    return res

In [60]:
class Config(object):
    n_layer = 4
    batch_size = 1024
    valid_size = 1024
    
    dim=3311; 
    Ntime=segs; 
    delta=1/Ntime
    sqrt_deltaT=np.sqrt(1.0/Ntime); 
    lam=1; 

    logging_frequency = 100
    verbose = True
    y_init_range = [0, 1]
    
    num_hiddens = [dim,64,64,1] ## 256 ,256
    
def get_config(name):
    try:
        return globals()[name]
    except KeyError:
        raise KeyError("config not defined.")

cfg=get_config('Config')

In [61]:
class Dense(nn.Module): 
    def __init__(self,cin, cout, batch_norm=False, activate=True): 
        super(Dense,self).__init__()
        self.cin=cin; 
        self.cout=cout; 
        self.activate=activate; 
        
        self.linear=nn.Linear(self.cin,self.cout) #The linear layer
        #BatchNorm1d: it requires the input to be a correct size
        if batch_norm: 
            self.bn=nn.BatchNorm1d(cout,eps=EPSILON,momentum=MOMENTUM)
        else: 
            self.bn=None
      #  nn.init.normal_(self.linear.weight,std=5.0/np.sqrt(cin+cout))
        # This is the He initialization
        
    def forward(self,x): 
        x=self.linear(x)
        if self.bn is not None:
            x=self.bn(x)
        if self.activate:
            x=torch.relu(x)
        return x 
    
class FFN(nn.Module):
    def __init__(self, config):
        super(FFN,self).__init__()
        self.config=config
        
        self.bn=nn.BatchNorm1d(config.num_hiddens[0],eps=EPSILON,momentum=MOMENTUM) ## So there is batch norm no problem
        # range(1,5): 1,2,3,4
        self.layers=[Dense(config.num_hiddens[i-1],config.num_hiddens[i]) for i in range(1, len(config.num_hiddens)-1)]
        self.layers+=[Dense(config.num_hiddens[-2], config.num_hiddens[-1],activate=False)]
        self.layers=nn.Sequential(*self.layers)
    
    def forward(self,x):
        #x=self.bn(x)
        x=self.layers(x)
        return x 
    
class Lookback_PPDE_Backward(nn.Module):
    def __init__(self,cfg): 
        super(Lookback_PPDE_Backward,self).__init__()
        self.cfg=cfg
        self.Ntime=self.cfg.Ntime 
        self.mList=nn.ModuleList([FFN(self.cfg) for _ in range(self.Ntime)])
        
    def forward(self,batch_sig,batch_dW,batch_YT, batch_x, batch_sel): 
        Y=batch_YT
        for i in np.arange(segs-1,-1, -1):
            Y=Y-Y*r*T/segs-sigma*torch.sum( self.mList[i](batch_sig[:,i,:])*batch_dW[:,i,:],axis=1,keepdim=True)
            gi=terminal_f(batch_x[:,:batch_sel[i],:])
            Y=torch.relu(gi-Y)+Y
        return Y

In [62]:
def loss_var(x):
    temp=torch.var(x)
    return temp

In [69]:
import torch.optim as optim
from torch.nn import Parameter
import math
model_PPDE_bw=Lookback_PPDE_Backward(cfg)
model_PPDE_bw#.to(device)
optimizer=optim.Adam(model_PPDE_bw.parameters(),lr=1e-4)
grad_clip=0.2
#scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=80, gamma=0.1)

In [70]:
y0_mean=[];
loss_vec=[];

## 0.5823 -- 0.5784 -- 0.5862

In [71]:
for i in range(100):
    batch_sig, batch_dw, batch_y,batch_x, batch_sel = generate_samples(batch_in=1000)

    x_temp=model_PPDE_bw(batch_sig,batch_dw,batch_y,batch_x, batch_sel)
    loss_temp=loss_var(x_temp)

    optimizer.zero_grad()
    loss_temp.backward()
    optimizer.step()
#   scheduler.step()
    y0_val=x_temp.mean().cpu().detach().numpy()
    loss_val=loss_temp.cpu().detach().numpy()
    
    y0_mean.append(y0_val)
    loss_vec.append(loss_val)
    if grad_clip: 
        nn.utils.clip_grad_value_(model_PPDE_bw.parameters(), grad_clip)
    
    print("Iter:", i, 'The mean Y0 is', y0_val , 'Variance is:' ,loss_val)

Iter: 0 The mean Y0 is 6.80053 Variance is: 82.21599
Iter: 1 The mean Y0 is 6.672508 Variance is: 85.75062
Iter: 2 The mean Y0 is 6.933611 Variance is: 86.79514
Iter: 3 The mean Y0 is 6.6699257 Variance is: 86.46731
Iter: 4 The mean Y0 is 6.232517 Variance is: 75.3734
Iter: 5 The mean Y0 is 6.3225865 Variance is: 69.89292
Iter: 6 The mean Y0 is 6.386982 Variance is: 83.58656
Iter: 7 The mean Y0 is 6.94478 Variance is: 96.37202
Iter: 8 The mean Y0 is 6.664698 Variance is: 88.417244
Iter: 9 The mean Y0 is 6.722926 Variance is: 86.04497
Iter: 10 The mean Y0 is 6.797552 Variance is: 71.551216
Iter: 11 The mean Y0 is 6.3591647 Variance is: 70.60002
Iter: 12 The mean Y0 is 7.045901 Variance is: 99.01353
Iter: 13 The mean Y0 is 6.544446 Variance is: 84.23046
Iter: 14 The mean Y0 is 6.989865 Variance is: 95.8579
Iter: 15 The mean Y0 is 6.6292787 Variance is: 88.07944
Iter: 16 The mean Y0 is 6.618153 Variance is: 79.13193
Iter: 17 The mean Y0 is 6.7711997 Variance is: 85.96969
Iter: 18 The mean

In [74]:
df=pd.DataFrame()
df['y_pred']=y0_mean
df['loss_var']=loss_vec

df_temp=df[11:]
lw=df_temp.loss_var.mean()-2.0*df_temp.loss_var.std()
up=df_temp.loss_var.mean()+2.0*df_temp.loss_var.std()
df1=df_temp[df_temp.loss_var>=lw]
df1=df1[df1.loss_var<=up]
df1.y_pred.mean()

6.706291647518382